In [1]:
import pandas as pd
import numpy as np
import ast

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

import spacy
from gensim import corpora
from gensim.models.ldamodel import LdaModel

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/matteo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
DATA_FOLDER = "data/preprocessed/"

movies = pd.read_csv(DATA_FOLDER + "preprocessed_movies.csv")

for col in movies.columns:
    try:
        movies[col] = movies[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    except:
        pass
    print(col, type(movies[col][0]))

wikipedia_id <class 'numpy.float64'>
freebase_id <class 'str'>
title <class 'str'>
languages <class 'float'>
countries <class 'list'>
genres <class 'list'>
keywords <class 'float'>
release_date <class 'numpy.int64'>
runtime <class 'numpy.float64'>
plot_summary <class 'str'>
cold_war_side <class 'str'>
character_western_bloc_representation <class 'list'>
character_eastern_bloc_representation <class 'list'>
western_bloc_values <class 'list'>
eastern_bloc_values <class 'list'>
theme <class 'list'>


In [18]:
def create_theme(df):
    new_df = df[["theme", 'cold_war_side', 'title', 'release_date']]
    # get only Western and Eastern movies
    new_df = new_df[(new_df["cold_war_side"] == "Western") | (new_df["cold_war_side"] == "Eastern")]
    new_df = new_df.rename(columns={'cold_war_side': 'movie_side'})
    new_df = new_df.drop_duplicates(subset=["theme"])
    # be sure theme is a list
    new_df["theme"] = new_df["theme"].apply(lambda x: x if isinstance(x, list) else [x])
    # be sure all elements of the list are strings
    new_df["theme"] = new_df["theme"].apply(lambda x: [str(i) for i in x])
    # use lower case for theme for each element of the list
    new_df["theme"] = new_df["theme"].apply(lambda x: [i.lower() for i in x])
    return new_df

theme_df = create_theme(movies)

In [21]:
theme_df.sample(5)

,theme,movie_side,title,release_date
3054,"[destruction of friendship, betrayal, the horr...",Eastern,Bullet in the Head,1990
18992,"[crime, serial killer, world war ii, police in...",Western,The Devil Strikes at Night,1957
18850,"[mystery, friendship, loyalty, secrecy, cold w...",Eastern,The Cutlass,1973
13112,[conflict between personal ambition and tradit...,Eastern,Obochina,1978
18890,"[war, sacrifice, female empowerment, courage, ...",Eastern,The Dawns Here Are Quiet,1972


In [22]:
# Preprocess the text
def preprocess(string_list, nlp, words_to_remove):
    docs = [nlp(s) for s in string_list]
    tokens = []
    for doc in docs:
        tokens.extend([token.lemma_ for token in doc if not token.is_stop and not token.is_punct and token.lemma_ not in words_to_remove])
    return tokens

def topic_detection(df, nb_topics, nb_passes):
    # Create a dictionary and corpus for Gensim
    dictionary = corpora.Dictionary(df)
    corpus = [dictionary.doc2bow(text) for text in df]
    
    # Train LDA model
    lda_model = LdaModel(corpus, num_topics=nb_topics, id2word=dictionary, passes=nb_passes)
    
    # Print the topics
    for idx, topic in lda_model.print_topics(-1):
        print(f'Topic: {idx}\nWords: {topic}\n')
    
    return lda_model, corpus, dictionary

# Assign topics to characters
def get_dominant_topic(lda_model, corpus):
    topics = []
    for row in corpus:
        row = lda_model[row]
        max_topic, max_val = 0, 0
        for (topic_num, prop_topic) in row:
            if prop_topic > max_val:
                max_val = prop_topic
                max_topic = topic_num
        topics.append(max_topic)
    return topics

In [23]:
nlp = spacy.load('en_core_web_sm')
words_to_remove = {'theme'}

In [30]:
theme_df['processed_repres'] = theme_df['theme'].apply(preprocess, args=(nlp, words_to_remove))
theme_topic = topic_detection(theme_df['processed_repres'], 5, 15)
theme_df['dominant_topic'] = get_dominant_topic(theme_topic[0], theme_topic[1])
print(theme_df['dominant_topic'].value_counts())

Topic: 0
Words: 0.057*"personal" + 0.044*"love" + 0.042*"struggle" + 0.035*"identity" + 0.028*"sacrifice" + 0.023*"cultural" + 0.019*"life" + 0.018*"conflict" + 0.015*"social" + 0.015*"family"

Topic: 1
Words: 0.067*"moral" + 0.041*"oppression" + 0.026*"dilemma" + 0.025*"struggle" + 0.023*"freedom" + 0.021*"betrayal" + 0.020*"conflict" + 0.020*"rebellion" + 0.020*"authority" + 0.020*"ambiguity"

Topic: 2
Words: 0.119*"war" + 0.045*"espionage" + 0.044*"sacrifice" + 0.034*"heroism" + 0.030*"cold" + 0.025*"conflict" + 0.023*"loyalty" + 0.021*"betrayal" + 0.021*"tension" + 0.019*"identity"

Topic: 3
Words: 0.069*"crime" + 0.064*"justice" + 0.035*"betrayal" + 0.032*"morality" + 0.026*"corruption" + 0.026*"revenge" + 0.022*"adventure" + 0.019*"investigation" + 0.018*"law" + 0.017*"heroism"

Topic: 4
Words: 0.027*"resilience" + 0.027*"dynamic" + 0.025*"family" + 0.023*"survival" + 0.020*"human" + 0.018*"social" + 0.017*"friendship" + 0.016*"age" + 0.015*"community" + 0.014*"adventure"

domina

In [110]:
character_eastern.sample(5)

,movie_side,title,year_release_date,character_name,character_representation,character_side,processed_repres,dominant_topic
13061,None,La Muerte de Mikel,1984,Mikel,"[begoña, doña maria luisa, martín, values of b...",Eastern,"[begoña, doña, maria, luisa, martín, basque, n...",3
26964,Western,The Odessa File,1974,Eduard Roschmann,"[former ss officer, power-driven, villain]",Eastern,"[ss, officer, power, drive, villain]",1
23315,Eastern,The Apple Game,1977,Czech society,"[satire, critical commentary]",Eastern,"[satire, critical, commentary]",1
12522,Eastern,Kievlyanka,1958,Gali,"[resilience, orphan archetype]",Eastern,"[resilience, orphan]",3
28989,None,The World of Suzie Wong,1960,Suzie Wong,"[young woman forced into prostitution, surviva...",Eastern,"[young, woman, force, prostitution, survivalis...",2


In [111]:
eastern_movie_eastern_char = character_eastern[character_eastern["movie_side"] == 'Eastern']
western_movie_eastern_char = character_eastern[character_eastern["movie_side"] == 'Western']
eastern_movie_western_char = character_western[character_western["movie_side"] == 'Eastern']
western_movie_western_char = character_western[character_western["movie_side"] == 'Western']

# count dominant topic for each df above
print("Eastern movie Eastern characters")
print(eastern_movie_eastern_char['dominant_topic'].value_counts())

print("Western movie Eastern characters")
print(western_movie_eastern_char['dominant_topic'].value_counts())

print("Eastern movie Western characters")
print(eastern_movie_western_char['dominant_topic'].value_counts())

print("Western movie Western characters")
print(western_movie_western_char['dominant_topic'].value_counts())

Eastern movie Eastern characters
dominant_topic
0    726
3    659
2    555
1    424
Name: count, dtype: int64
Western movie Eastern characters
dominant_topic
1    752
3    396
0    240
2    223
Name: count, dtype: int64
Eastern movie Western characters
dominant_topic
2    308
0    248
1    246
3    104
Name: count, dtype: int64
Western movie Western characters
dominant_topic
3    865
2    781
1    568
0    560
Name: count, dtype: int64


In [26]:
# plot topics
import pyLDAvis
from pyLDAvis import gensim_models

In [31]:
# pyLDAvis.display(gensim_models.prepare(western_char_topic[0], western_char_topic[1], western_char_topic[2]))
pyLDAvis.display(gensim_models.prepare(theme_topic[0], theme_topic[1], theme_topic[2]))

In [74]:
pyLDAvis.display(gensim_models.prepare(eastern_char_topic[0], eastern_char_topic[1], eastern_char_topic[2]))

In [243]:
from empath import Empath
lexicon = Empath()

# Select lexical categories of interest
CAT = ['politics', 'leader', 'military', 'heroic', 'law', 'affection', 'help',
        'pride', 'family', 'love', 'power', 'deception', 'friends', 'sadness', 'disappointment']

# Define groups of similar topics
TOPIC_GROUPS = {
    'love&affection': ['love', 'affection', 'family', 'friends'],
    'politics&power': ['politics', 'leader', 'military', 'power'],
    'deception&sadness': ['deception', 'sadness', 'disappointment']
}

def semantical_analysis(text, categories):
    """
    Perform a semantical analysis of a text using the Empath lexicon

    Parameters
    text : str, the text to analyze
    categories : list[str], the lexical categories to consider

    Returns
    dict[str, float], the analysis results (only the categories with non-zero values)
    """
    analysis = lexicon.analyze(text, categories=categories, normalize=True)
    return None if max(analysis.values()) == 0 else {k: v for k, v in analysis.items() if v != 0}

def map_topic_to_group(topic, groups):
    """
    Map a topic to a group of similar topics
    """
    for group, topics in groups.items():
        if topic in topics:
            return group
    return topic  

In [244]:
for cat in list(lexicon.cats.keys()):
    print(cat)

help
office
dance
money
wedding
domestic_work
sleep
medical_emergency
cold
hate
cheerfulness
aggression
occupation
envy
anticipation
family
vacation
crime
attractive
masculine
prison
health
pride
dispute
nervousness
government
weakness
horror
swearing_terms
leisure
suffering
royalty
wealthy
tourism
furniture
school
magic
beach
journalism
morning
banking
social_media
exercise
night
kill
blue_collar_job
art
ridicule
play
computer
college
optimism
stealing
real_estate
home
divine
sexual
fear
irritability
superhero
business
driving
pet
childish
cooking
exasperation
religion
hipster
internet
surprise
reading
worship
leader
independence
movement
body
noise
eating
medieval
zest
confusion
water
sports
death
healing
legend
heroic
celebration
restaurant
violence
programming
dominant_heirarchical
military
neglect
swimming
exotic
love
hiking
communication
hearing
order
sympathy
hygiene
weather
anonymity
trust
ancient
deception
fabric
air_travel
fight
dominant_personality
music
vehicle
politeness
t

In [245]:
# Apply the analysis to each row
character_df['empath_analysis'] = character_df['character_representation'].apply(lambda x: semantical_analysis(' '.join(x), CAT))
character_empath_df = character_df.dropna(subset=['empath_analysis']).copy()

print('Total number of characters :', len(character_df))
print('Characters with empath analysis not None :', len(character_empath_df))

print('\nNumber of characters per category :')
for cat in CAT:
    cat_count = character_empath_df['empath_analysis'].apply(lambda x: cat in x.keys()).sum()
    print(f'{cat}: {cat_count}')

character_empath_df.sample(5)

Total number of characters : 8243
Characters with empath analysis not None : 6588

Number of characters per category :
politics: 814
leader: 1191
military: 724
heroic: 3896
law: 948
affection: 711
help: 512
pride: 1915
family: 269
love: 554
power: 1097
deception: 604
friends: 359
sadness: 626
disappointment: 215


,movie_side,title,year_release_date,character_name,character_representation,character_side,empath_analysis
23296,Western,The Angry Red Planet,1959,Dr. Iris Ryan,"[colonel tom o'bannion, bravery, scientific in...",Western,"{'military': 0.125, 'heroic': 0.25}"
14399,Western,Macao,1952,Julie Benson,"[cynical, sultry night club singer, femme fata...",Eastern,{'heroic': 0.125}
2186,Western,Ashanti,1979,David Linderby,"[values freedom, fight against injustice, arch...",Western,"{'heroic': 0.14285714285714285, 'power': 0.142..."
22470,None,Suspect,1960,Kathleen Riley,"[public defender, justice, moral integrity, hero]",Western,"{'military': 0.16666666666666666, 'heroic': 0...."
24552,Western,The Divided Heart,1954,Natural Mother,"[loss, survival, family, victim]",Eastern,"{'family': 0.25, 'sadness': 0.25}"


In [251]:
def get_empath_evolution(df, empath_column, year_column='year_release_date'):
    """
    Create a DataFrame to track the evolution of empath topics across years
    """
    empath_topics = set()
    for entry in df[empath_column]:
        empath_topics.update(entry.keys())

    # Create a new DataFrame to track topic proportions across years
    empath_evolution = {topic: [] for topic in empath_topics}
    empath_evolution['year'] = []

    for year in sorted(df[year_column].unique()):
        year_data = df[df['year_release_date'] == year][empath_column]
        aggregated = {topic: 0 for topic in empath_topics}
        count = len(year_data)

        for row in year_data:
            for topic, value in row.items():
                aggregated[topic] += value

        if count > 0:
            for topic in aggregated:
                aggregated[topic] /= count

        for topic, value in aggregated.items():
            empath_evolution[topic].append(value)

        empath_evolution['year'].append(year)

    return pd.DataFrame(empath_evolution)

empath_evolution_df = get_empath_evolution(character_empath_df, 'empath_analysis')
melted_empath_df = empath_evolution_df.melt(id_vars=['year'], var_name='topic', value_name='proportion')

# group similar lexicon categories
melted_empath_df['topic'] = melted_empath_df['topic'].apply(map_topic_to_group, groups=TOPIC_GROUPS)
melted_empath_df = melted_empath_df.groupby(['year', 'topic'], as_index=False).agg({'proportion': 'sum'})

In [262]:
fig = go.Figure()

# set topic color palette
unique_topics = melted_empath_df['topic'].unique()
color_palette = px.colors.qualitative.Plotly
topic_colors = {topic: color_palette[i] for i, topic in enumerate(unique_topics)}

# Map colors directly in the DataFrame
melted_empath_df['color'] = melted_empath_df['topic'].map(topic_colors)

fig = px.line(
    melted_empath_df,
    x='year',
    y='proportion',
    color='topic',
    title='Evolution of Grouped Empath Topics Across Years',
    color_discrete_map=topic_colors
)

fig.update_layout(
    width = 1100,
    height = 600,
    xaxis_title="Movie Release Year",
    yaxis_title="Frequency of a Category",
    legend_title="Categories",
    title={
        "text": "Changing Themes in Character Representation Across Years",
        "x": 0.5,
        "xanchor": "center",
        "font": {"size": 20, "family": "Arial", "weight": "bold"}
    },
    margin=dict(l=50, r=50, t=50, b=50),  # Adjust margins
)

fig.show()

In [263]:
fig = go.Figure()

unique_years = sorted(melted_empath_df['year'].unique())
max_y = melted_empath_df['proportion'].max()

for year in unique_years:
    year_data = melted_empath_df[melted_empath_df['year'] == year]
    grouped_data = year_data.groupby('topic', as_index=False).agg({'proportion': 'sum'})
    
    fig.add_trace(go.Bar(
        x=grouped_data['topic'],
        y=grouped_data['proportion'],
        name=f"Year {year}",
        visible=(year == unique_years[0]),  # Only show the first year's data initially
        marker=dict(color=[topic_colors[topic] for topic in grouped_data['topic']])
    ))

# Define steps for the slider
steps = []
for i, year in enumerate(unique_years):
    step = dict(
        method="update",
        args=[
            {"visible": [j == i for j in range(len(unique_years))]},  # Toggle visibility for the selected year
            {"title": f"Empath Topic Proportions for Year: {year}"}
        ],
        label=str(year)
    )
    steps.append(step)

# Add the slider to the layout
fig.update_layout(
    sliders=[{
        "active": 0,
        "currentvalue": {"prefix": "Year: "},
        "pad": {"t": 50},
        "steps": steps
    }],
    title={
        "text": "Characters' Empath Features Distribution by Year",
        "x": 0.5,
        "xanchor": "center",
        "font": {"size": 20, "family": "Arial", "weight": "bold"}
    },
    xaxis_title="Empath Topics",
    yaxis_title="Proportion",
    yaxis=dict(range=[0, max_y]),  # Set the fixed y-axis range
    legend_title="Year",
    barmode="group"  # This ensures bars are grouped by topics
)

fig.show()
